In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras import *
from tensorflow.keras.optimizers import Adam
import optuna as op
import pickle
import matplotlib.pyplot as plt
import json
import plotly
print("TensorFlow version:", tf.__version__)
print(tf.config.list_physical_devices('GPU'))


TensorFlow version: 2.7.0
[]


In [3]:
dataset = pd.read_csv("./../../datasets/All.csv")

# shuffle rows
dataset = dataset.sample(frac=1).reset_index(drop=True)

# IMPOR 
X = pd.DataFrame()
for col in ["Hour", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday", "SchoolHoliday", "Janurary", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]:
    X[col] = dataset[col].astype(float)
X_hash = pd.util.hash_pandas_object(X, index=False)

Y = pd.DataFrame()
for col in ["NO2-ug-per-m3"]:
    Y[col] = dataset[col].astype(float)


# split into train and test
train_quota = 0.8
X_train = X[:int(len(X) * train_quota)]
X_test = X[int(len(X) * train_quota):]
Y_train = Y[:int(len(Y) * train_quota)]
Y_test = Y[int(len(Y) * train_quota):]

print(X.head())
print(Y.head())



   Hour  Monday  Tuesday  Wednesday  Thursday  Friday  Saturday  Sunday  \
0   9.0     0.0      0.0        0.0       0.0     0.0       0.0     1.0   
1  19.0     0.0      0.0        0.0       0.0     1.0       0.0     0.0   
2  14.0     0.0      1.0        0.0       0.0     0.0       0.0     0.0   
3  12.0     0.0      0.0        1.0       0.0     0.0       0.0     0.0   
4  11.0     0.0      0.0        0.0       0.0     0.0       0.0     1.0   

   SchoolHoliday  Janurary  ...  March  April  May  June  July  August  \
0            1.0       0.0  ...    0.0    0.0  0.0   0.0   0.0     0.0   
1            1.0       0.0  ...    0.0    0.0  0.0   0.0   0.0     0.0   
2            1.0       0.0  ...    0.0    0.0  0.0   0.0   0.0     0.0   
3            1.0       0.0  ...    0.0    0.0  0.0   0.0   0.0     0.0   
4            1.0       0.0  ...    0.0    0.0  0.0   0.0   0.0     0.0   

   September  October  November  December  
0        0.0      0.0       0.0       0.0  
1        0.0    

In [4]:
# set trining duration
epochs_per_batch = 3

def std(model):
    stds = []
    predictions = model.predict(X_test.to_numpy())
    predictions = np.abs(predictions - Y_test.to_numpy()) / Y_test.to_numpy()
    for guesses in predictions[0, :]:
        guesses = np.sort(predictions[:, 0])
        stds.append(guesses[int(len(guesses) * 0.682)])

    std = np.mean(np.array(stds))

    return std

# 1. Define an objective function to be maximized.
def objective(trial):

    activations = ['relu', 'linear', 'sigmoid', 'tanh', 'softmax', 'softsign', 'selu', 'elu', 'exponential', 'softplus']
    
    model = tf.keras.Sequential()
    model.add(Dense(X.shape[1], activation=trial.suggest_categorical('layer_in_activation', activations)))

    # 2. hyper parameters
    batch_size = trial.suggest_int("batch_size", 5, 2000)
    epochs = epochs_per_batch * batch_size
    learning_rate = trial.suggest_float("learning_rate", 0.00001, 1.0, log=True)
    
    model.add(Dense(trial.suggest_int("layer_1_neurons", 0, 300), activation=trial.suggest_categorical('layer_1_activation', activations)))
    model.add(Dense(trial.suggest_int("layer_2_neurons", 0, 300), activation=trial.suggest_categorical('layer_2_activation', activations)))
    model.add(Dense(Y.shape[1], activation=trial.suggest_categorical('layer_out_activation', activations)))

    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='mse')


    model.fit(
        X_train.to_numpy(),
        Y_train.to_numpy(),
        batch_size=batch_size,
        epochs=epochs,
        validation_split=0.0,
        verbose=0
    )

    return std(model)
    

In [6]:
# Get study name
X_hash = pd.util.hash_pandas_object(X, index=False).to_numpy().sum()
Y_hash = pd.util.hash_pandas_object(Y, index=False).to_numpy().sum()
study_name = "Optunia_study_"+str(epochs_per_batch)+"_"+str(X_hash)+"_"+str(Y_hash)+".pickle"

# Load Study
study = None
try:
    with open(study_name, 'rb') as f:
        study = pickle.load(f)
        print("Loaded study", study_name)
        print("Best trial:", study.best_trial)
except:
    study = op.create_study(direction='minimize')
    print("Created study", study_name)

Loaded study Optunia_study_3_11800395523791420690_18317216514516929911.pickle
Best trial: FrozenTrial(number=137, values=[0.41968310156533883], datetime_start=datetime.datetime(2021, 12, 3, 19, 23, 53, 824800), datetime_complete=datetime.datetime(2021, 12, 3, 19, 25, 49, 844477), params={'layer_in_activation': 'selu', 'batch_size': 1349, 'learning_rate': 0.0013238035776299386, 'layer_1_neurons': 282, 'layer_1_activation': 'softplus', 'layer_2_neurons': 190, 'layer_2_activation': 'exponential', 'layer_out_activation': 'elu'}, distributions={'layer_in_activation': CategoricalDistribution(choices=('relu', 'linear', 'sigmoid', 'tanh', 'softmax', 'softsign', 'selu', 'elu', 'exponential', 'softplus')), 'batch_size': IntUniformDistribution(high=2000, low=5, step=1), 'learning_rate': LogUniformDistribution(high=1.0, low=1e-05), 'layer_1_neurons': IntUniformDistribution(high=300, low=0, step=1), 'layer_1_activation': CategoricalDistribution(choices=('relu', 'linear', 'sigmoid', 'tanh', 'softmax

In [7]:
op.visualization.plot_optimization_history(study)       

In [8]:
df = study.trials_dataframe()

# get row with lowes "value" from df
best_trial = df.loc[df['value'].idxmin()]
best_trials_params = best_trial.to_dict()
# iterate each parameter and print it
for key, value in best_trials_params.items():
    # pad key with spaces
    key = key.ljust(30)
    print(key, ":", value)

number                         : 137
value                          : 0.41968310156533883
datetime_start                 : 2021-12-03 19:23:53.824800
datetime_complete              : 2021-12-03 19:25:49.844477
duration                       : 0 days 00:01:56.019677
params_batch_size              : 1349
params_layer_1_activation      : softplus
params_layer_1_neurons         : 282
params_layer_2_activation      : exponential
params_layer_2_neurons         : 190
params_layer_in_activation     : selu
params_layer_out_activation    : elu
params_learning_rate           : 0.0013238035776299386
state                          : COMPLETE


In [9]:
op.visualization.plot_slice(study)

In [10]:
op.visualization.plot_contour(study, params=['layer_1_neurons', 'layer_2_neurons'])

In [9]:
# optimize hyperparameters

while True:
    study.optimize(objective, n_trials=1)
    with open(study_name, 'wb') as f:
        pickle.dump(study, f)

[I 2021-12-07 11:04:50,738] Trial 142 finished with value: 0.49495406377883183 and parameters: {'layer_in_activation': 'selu', 'batch_size': 1465, 'learning_rate': 0.002093368881844366, 'layer_1_neurons': 292, 'layer_1_activation': 'softplus', 'layer_2_neurons': 211, 'layer_2_activation': 'exponential', 'layer_out_activation': 'elu'}. Best is trial 137 with value: 0.41968310156533883.
[I 2021-12-07 11:06:27,346] Trial 143 finished with value: 0.4785620199667441 and parameters: {'layer_in_activation': 'selu', 'batch_size': 1402, 'learning_rate': 0.0018098947695644287, 'layer_1_neurons': 286, 'layer_1_activation': 'softplus', 'layer_2_neurons': 192, 'layer_2_activation': 'exponential', 'layer_out_activation': 'elu'}. Best is trial 137 with value: 0.41968310156533883.


KeyboardInterrupt: 

In [18]:
# Make a model from the best trial and train it longer#

def get_model(trial_row):
    model = tf.keras.Sequential()
    model.add(Dense(X.shape[1], trial_row['params_layer_in_activation']))
    
    model.add(Dense(trial_row['params_layer_1_neurons'], activation=trial_row['params_layer_1_activation']))
    model.add(Dense(trial_row['params_layer_2_neurons'], activation=trial_row['params_layer_2_activation']))
    model.add(Dense(Y.shape[1], activation=trial_row['params_layer_out_activation']))

    model.compile(
        optimizer=Adam(learning_rate=trial_row['params_learning_rate']),
        loss='mse')

    return model

trial_row = study.trials_dataframe().loc[study.best_trial.number]
model = get_model(trial_row)
model.fit(
        X_train.to_numpy(),
        Y_train.to_numpy(),
        batch_size=trial_row['params_batch_size'],
        epochs=trial_row['params_batch_size']*3,
        validation_split=0.0,
        verbose=0
    )

print(std(model))

0.47258663177490234
